# Configure a digital twin: Cloud harvester
This is a KWH configuration application which allows to configure a simple digital twin based on cloud approach using python reference implementation and S³I python library. As a result, you will get a configuration file and a credentials file in the configs folder. With the following example, we will show you how to register the identify of a **cloud harvester** and create the respective configuration file using this app.  
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I and modelling language library.

In [ ]:
import s3i
import jwt
import getpass
import json 
import os
import ml
from tools import print_with_timestamp, yes, no, configuration_app_id, configuration_app_secret

In [ ]:
%%html
<img src="harvester_images/folie_1_1.png", width=1000, height=1000>

## Configure the application
In order to register a thing identify for the harvester, we need a connection with S³I. This means, this app needs a client id and the respective secret to authenticate itself at the S³I. The client credentials are already prepared for you. You only need to enter your username and password in the following input fields.

In [ ]:
print_with_timestamp("KWH application to configure a digital twin, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=configuration_app_id,
                                             client_secret=configuration_app_secret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token'''
parsed_username = jwt.decode(access_token, verify=False)["preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

## Path of configuration files

In [ ]:
config_path = os.path.abspath(os.path.join("", "configs"))

## Create an identity for your cloud harvester
In the following, a thing identify is created using the S³I Config API. Meanwhile, we also create a repository entry for the cloud harvester in the S³I Repository. 

In [ ]:
s3i_config = s3i.Config(token=access_token)
thing_name = input("[S³I]: Please name your Harvester(e.g. my_cloud_harvester): ")

"""Create the thing identity"""
resp = s3i_config.create_thing()
thing_id = resp.json().get("identifier", None)
thing_secret = resp.json().get("secret", None)
print_with_timestamp("{} has the identifier: {}".format(thing_name, thing_id))
print_with_timestamp("{} has the respective secret: {}".format(thing_name, thing_secret))

"""Create a repo entry"""
s3i_config.create_cloud_copy(thing_id=thing_id)

"""Store the credentials"""
cred_filepath = os.path.join(config_path, "{}_cred.json".format(thing_name))
with open(cred_filepath, 'wb') as file:
    file.write(json.dumps(resp.json()).encode('utf-8'))

## Make the configuration file 
As next we introduce how to simply configure your digital twin of harvester using python reference implementation library. The respective fml40 model of harvester is shown as follow. 

In [ ]:
%%html
<img src="harvester_images/cloud_h.png", width=1000, height=1000>

In [ ]:
config_file_name = ml.make_thing_config(thing_id=thing_id, name=thing_name, 
                                     roles=[{"class": "fml40::Harvester"}],
                                     features=[{"class": "ml40::RotationalSpeed", "rpm": 2005}], config_path=config_path)

thing_model = ml.load_config(config_filepath=os.path.join(config_path, config_file_name))

print_with_timestamp("The fml40 model of the created harvester in the form of JSON: " + json.dumps(thing_model, indent=2))
print_with_timestamp("The created configuration file is stored in the folder 'configs'.")